In [1]:
# !pip install langchain_community --quiet

In [2]:
import json
import pandas as pd
from langchain.sql_database import SQLDatabase
from langchain_community.utilities import SQLDatabase
from sqlalchemy import create_engine

In [4]:
with open("/home/ubuntu/config.json") as file:
    config = json.load(file)

database_uri = "postgresql://{user}:{password}@{host}:{port}/{database}".format(**config["rds_connect"])

In [5]:
rds_engine = create_engine(url=database_uri)

In [6]:
df = pd.read_csv("hotel-recommendations-cleaned.csv")
# df.head()
# df.to_sql?

In [7]:
%%time
df.to_sql(
    name="hoteldata",  # table_name
    con=rds_engine, 
    if_exists="replace",
    index=False
)

CPU times: user 576 ms, sys: 14.2 ms, total: 590 ms
Wall time: 1.53 s


907

In [8]:
rds_db = SQLDatabase.from_uri(database_uri=database_uri)
print(rds_db.dialect)
print(rds_db.get_usable_table_names())

postgresql
['hoteldata']


In [9]:
print(rds_db.get_table_info())


CREATE TABLE hoteldata (
	room_type TEXT, 
	onsite_rate DOUBLE PRECISION, 
	room_amenities TEXT, 
	max_occupancy BIGINT, 
	roomdescription TEXT, 
	hotel_name TEXT, 
	city TEXT, 
	country TEXT, 
	star_rating BIGINT, 
	meals_included BOOLEAN
)

/*
3 rows from hoteldata table:
room_type	onsite_rate	room_amenities	max_occupancy	roomdescription	hotel_name	city	country	star_rating	meals_included
Vacation Home	636.09	Air conditioning: ;Closet: ;Fireplace: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Heating: ;High cha	4	Shower, Kitchenette, 2 bedrooms, 1 double bed or 2 single beds	Pantlleni	Beddgelert	United Kingdom	3	False
Vacation Home	591.74	Air conditioning: ;Closet: ;Dishwasher: ;Fireplace: ;Free Wi-Fi in all rooms!: ;Full kitchen: ;Heati	4	Shower, Kitchenette, 2 bedrooms, 1 double bed or 2 single beds	Willow Cottage	Beverley	United Kingdom	3	False
Guest room, Queen or Twin/Single Bed(s)	0.0	None	2	None	AC Hotel Manchester Salford Quays	Manchester	United Kingdom	4	False
*/
